In [4]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
drive.flush_and_unmount()
ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # we mount the google drive at /content/drive

Drive not mounted, so nothing to flush and unmount.
/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%cd /content/drive/My\ Drive/Colab\ stuff/sensor_anomaly/

/content/drive/My Drive/Colab stuff/sensor_anomaly


In [6]:
import pandas as pd
import numpy as np

In [ ]:
def process_activity_column(column):
    change_str = ""
    changing = False
    processed_column = column.copy()
    max_stack = [0]
    max_stack_index = []
    for index, state in enumerate(column):
        if str(state).split()[-1] == 'begin':
            changing = True
#             print("Begin Found")
            if not change_str == '':# and (state.strip().split()[0] not in change_str.split('-')):
                change_str += '-'+ state.strip().split()[0]
            else:
                change_str = state.strip().split()[0]
#             print(index, change_str)
        if changing:
            processed_column[index] = change_str.split('-')
            if len(processed_column[index]) > max_stack[0]:
                max_stack = [len(processed_column[index])]
                max_stack_index = [index]
            elif len(processed_column[index]) == max_stack[0]:
                max_stack.append(len(processed_column[index]))
                max_stack_index.append(index)
        if str(state).split()[-1] == 'end':
            processed_column[index] = change_str.split('-')
#             print("End Found")
            end_act = state.strip().split()[0]
#             print("End act: ", end_act)
            acts = change_str.split('-')
            acts.remove(end_act)
            if not acts:
#                 print("no acts")
                changing = False
            change_str = '-'.join(acts)
#             print(index, acts, change_str)
    return processed_column, max_stack, max_stack_index

In [ ]:
data_milan = pd.read_csv("dataset/milan/data", sep = "\t", index_col=0, parse_dates=[0], infer_datetime_format=True, skip_blank_lines = True, names = ['Timestamp (UTC)', 'sensorID', 'state', 'activity'])
data_milan = data_milan.sort_index()
processed_column, max_stack, max_stack_index = process_activity_column(list(data_milan['activity']))
data_milan['mod_col'] = processed_column
data_milan['mod_col'] = data_milan['mod_col'].fillna('None')

In [ ]:
print(max_stack, max_stack_index)
print(data_milan.iloc[2661])
print(data_milan.index[0] + pd.Timedelta(minutes=15))

In [ ]:
def process_subset(df, uni_list_beg, cols):
    act_col = df['mod_col'].tolist()
    uni_list = []
    for i in act_col:
        uni_list += ['Other'] if i == 'None' else i
    uni_list = set(uni_list)
    act_col_beg_end = df['activity'].tolist()
    for index, state in enumerate(act_col_beg_end):
        if str(state).split()[-1] == 'begin':
            uni_list_beg.append(state.strip().split()[0])
        elif str(state).split()[-1] == 'end' and (str(state).split()[0] in uni_list_beg):
            uni_list_beg.remove(str(state).split()[0])
    uni_list = list(uni_list)
    uni_list.sort()
    sensorID = df['sensorID'].tolist()
    sensor_list = [0] * (len(cols)-1)
    for index, sensor in enumerate(cols[:-1]):
        sensor_list[index] = int(sensor in sensorID)
    return uni_list, uni_list_beg, sensor_list

In [ ]:
cols = data_milan.sensorID.unique()
#print(list(cols))
cols = list(cols)
cols.append('Activities')
df_sensors = pd.DataFrame(columns = cols)
df_sensors.head(5)

uni_list_beg = []
startTime = data_milan.index[0]
stepTime = pd.Timedelta(seconds=60)
endTime = data_milan.index[-1]
count = 0
countc = 0
while startTime  <= endTime:
    df_15mins = data_milan.loc[startTime: startTime + stepTime]
    #print(df_15mins)
    startTime = startTime + stepTime
#     print(startTime)
    uni_list, uni_list_beg, sensor_list =  process_subset(df_15mins, uni_list_beg, cols)
#     print("count: ", count)
#     print("List of open activities:", uni_list_beg)
#     print("List of activities:", uni_list)
    #print(list(cols))
    count+=1
    #print("Sensor Feature:", sensor_list)
    if (all(v == 0 for v in sensor_list) and uni_list_beg) or uni_list:
#         print("countc: ", countc)
        countc+=1
        if all(v == 0 for v in sensor_list):
            uni_list = list(uni_list_beg)
        list2append = uni_list
        sensor_list.append(list2append)
        sensor_series = pd.Series(sensor_list, index= df_sensors.columns)      
#         print(list(sensor_series))
        df_sensors = df_sensors.append(sensor_series, ignore_index=True)
    elif (all(v == 0 for v in sensor_list) and not uni_list_beg):
#         print("countc: ", countc)
        countc+=1
        sensor_list.append(['Other'])
        sensor_series = pd.Series(sensor_list, index= df_sensors.columns)      
#         print(list(sensor_series))
        df_sensors = df_sensors.append(sensor_series, ignore_index=True)

In [ ]:
# pd.set_option('display.max_rows', df_sensors.shape[0]+1)
# df_sensors

In [7]:
df_sensors.to_csv('dataset/milan/data_processed.csv', index = False)

In [48]:
df_sensors = pd.read_csv("dataset/milan/data_processed.csv", header = [0])
df_unique_rows = df_sensors.iloc[:,:-1]
df_unique_rows = df_unique_rows.drop_duplicates()
df_unique_rows.reset_index(drop=True, inplace=True)
dictSeries = df_unique_rows.to_dict('index')

In [49]:
print(len(df_unique_rows))
print(df_sensors.head(5))

4858
   M017  M009  M019  M020  T001  T002  ...  D001  M004  D003  D002  M024  Activities
0     1     1     1     0     0     0  ...     0     0     0     0     0   ['Other']
1     0     0     0     0     0     0  ...     0     0     0     0     0   ['Other']
2     0     0     0     0     0     0  ...     0     0     0     0     0   ['Other']
3     0     0     0     0     0     0  ...     0     0     0     0     0   ['Other']
4     0     0     0     0     0     0  ...     0     0     0     0     0   ['Other']

[5 rows x 34 columns]


In [50]:
key_list = list(dictSeries.keys())
val_list = list(dictSeries.values())
col_label = df_sensors.apply(lambda row: key_list[val_list.index(pd.Series(row[:-1]).to_dict())], axis = 1)
col_label = col_label.rename("Sensor_label")
df_sensors = df_sensors.join(col_label)
df_sensors

,M017,M009,M019,M020,T001,T002,M016,M015,M022,M012,M003,M005,M006,M008,M011,M028,M021,M025,M013,M010,M023,M014,M026,M018,M007,M027,M002,M001,D001,M004,D003,D002,M024,Activities,Sensor_label
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,['Other'],0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,['Other'],1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,['Other'],1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,['Other'],1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,['Other'],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118843,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,1,0,0,0,0,"['Kitchen_Activity', 'Other']",498
118844,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,1,1,0,0,0,0,['Other'],4857
118845,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,['Other'],22
118846,0,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,1,1,0,0,0,0,['Other'],4857


In [51]:
print(len(df_sensors.Sensor_label.unique()))

4858


In [59]:
for x in df_sensors['Activities']:
  print(*str(x))
# unique_data = [list(x) for x in set(tuple(x) for x in df_sensors['Activities'])]
# print(unique_data)
# dictSeries = {}
# for index,ele in enumerate(unique_data):
#     dictSeries[index] = ele

[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h e r ' ]
[ ' O t h 

KeyboardInterrupt: ignored

In [30]:
key_list = list(dictSeries.keys())
val_list = list(dictSeries.values())
print(val_list)
col_label = df_sensors['Activities'].map(lambda row: key_list[val_list.index(row)])
col_label = col_label.rename("activity_label")
df_sensors = df_sensors.join(col_label)

[['[', "'", 'K', 'i', 't', 'c', 'h', 'e', 'n', '_', 'A', 'c', 't', 'i', 'v', 'i', 't', 'y', "'", ',', ' ', "'", 'R', 'e', 'a', 'd', "'", ']'], ['[', "'", 'K', 'i', 't', 'c', 'h', 'e', 'n', '_', 'A', 'c', 't', 'i', 'v', 'i', 't', 'y', "'", ',', ' ', "'", 'O', 't', 'h', 'e', 'r', "'", ',', ' ', "'", 'R', 'e', 'a', 'd', "'", ']'], ['[', "'", 'D', 'e', 's', 'k', '_', 'A', 'c', 't', 'i', 'v', 'i', 't', 'y', "'", ',', ' ', "'", 'G', 'u', 'e', 's', 't', '_', 'B', 'a', 't', 'h', 'r', 'o', 'o', 'm', "'", ',', ' ', "'", 'O', 't', 'h', 'e', 'r', "'", ']'], ['[', "'", 'G', 'u', 'e', 's', 't', '_', 'B', 'a', 't', 'h', 'r', 'o', 'o', 'm', "'", ',', ' ', "'", 'M', 'o', 'r', 'n', 'i', 'n', 'g', '_', 'M', 'e', 'd', 's', "'", ',', ' ', "'", 'O', 't', 'h', 'e', 'r', "'", ']'], ['[', "'", 'M', 'a', 's', 't', 'e', 'r', '_', 'B', 'e', 'd', 'r', 'o', 'o', 'm', '_', 'A', 'c', 't', 'i', 'v', 'i', 't', 'y', "'", ',', ' ', "'", 'R', 'e', 'a', 'd', "'", ']'], ['[', "'", 'D', 'i', 'n', 'i', 'n', 'g', '_', 'R', 'm'

ValueError: ignored

In [ ]:
# df_sensors

In [ ]:
print(len(df_sensors.activity_label.unique()))
df_sensors.reset_index(drop=True, inplace=True)

In [ ]:
len_sentence = 4
startIndex = df_sensors.index[0]
stepIndex = 5
endIndex = df_sensors.index[-1]
with open("Sensor-Activity.txt", "w") as file:
    while startIndex + len_sentence <= endIndex:
        s = pd.Series(df_sensors.Sensor_label.loc[startIndex: startIndex + len_sentence])
        s = s.apply(str)
        sensor_sentence = s.str.cat(sep = " ")
#         print(sensor_sentence)
        s = pd.Series(df_sensors.activity_label.loc[startIndex: startIndex + len_sentence])
        s = s.apply(str)
        activity_sentence = s.str.cat(sep = " ")
#         print(activity_sentence)

        file.write(sensor_sentence + "\t" + activity_sentence + "\n")
        startIndex = startIndex + stepIndex